In [3]:
import os

import ifcopenshell
from ifcopenshell.util.element import get_psets

In [6]:
os.getcwd()

'C:\\Users\\naumo\\PycharmProjects\\IFC_test\\notebooks'

In [7]:
import pickle
G = pickle.load(open('../src/new_AR.pickle', 'rb'))

In [21]:
from src.ifc_to_nx_explorer import IfcToNxExplorer

node_attributes = IfcToNxExplorer.node_attributes


In [9]:
model = ifcopenshell.open('../Amundsena_IFC_24/АР_Амундсена_Син_R22.ifc')

ifcopenshell.file.file

In [20]:
classes = (
    'IfcWall',
    'IfcCurtainWall',
    'IfcDoor',
    'IfcWindow',
    'IfcBuildingElementProxy',
    'IfcSlab',
    "IfcFlowTerminal",
    "IfcFurniture",
     # 'IfcCovering',
     # 'IfcStair',
     # 'IfcRailing',
     # 'IfcSpace',
)

for cls in classes:
    l = model.by_type(cls)
    if len(l) > 0:
        print(cls, l[0].Name)


IfcWall Базовая стена:Внутренние - Кирпичная перегородка толщиной 250мм:301422
IfcCurtainWall Витраж:ADSK_Витраж_Без разрезки_Импосты 50х100:1188145
IfcBuildingElementProxy Кухонная плита:610 x 610 мм:516522
IfcSlab Перекрытие:Плитка на растворе=60мм:704022
IfcFlowTerminal раковина:раковина:505216
IfcFurniture Base - Double Door for sink:600 x 900 x 900mm:521321


In [32]:
build = model.by_type("IfcBuilding")[0]
build.get_info()

{'id': 142,
 'type': 'IfcBuilding',
 'GlobalId': '3HmrpkGhHBzfnu9Ed9Y4vm',
 'OwnerHistory': #42=IfcOwnerHistory(#39,#5,$,.NOCHANGE.,$,$,$,1698128163),
 'Name': '',
 'Description': None,
 'ObjectType': None,
 'ObjectPlacement': #33=IfcLocalPlacement(#358,#32),
 'Representation': None,
 'LongName': '',
 'CompositionType': 'ELEMENT',
 'ElevationOfRefHeight': None,
 'ElevationOfTerrain': None,
 'BuildingAddress': #138=IfcPostalAddress($,$,$,$,('Многоэтажный жилой дом со встроенными помещениями общественного назначения и подземной автостоянкой, расположенный по адресу: г. Екатеринбург, ул. Амундсена, 57а'),$,'','Москва','','Россия')}

In [34]:
walls = model.by_type("IfcWall")
element = walls[0]
element.get_info()

{'id': 507,
 'type': 'IfcWall',
 'GlobalId': '1syOIXGHX7Hg23ysDGAt9W',
 'OwnerHistory': #42=IfcOwnerHistory(#39,#5,$,.NOCHANGE.,$,$,$,1698128163),
 'Name': 'Базовая стена:Внутренние - Кирпичная перегородка толщиной 250мм:301422',
 'Description': None,
 'ObjectType': 'Базовая стена:Внутренние - Кирпичная перегородка толщиной 250мм',
 'ObjectPlacement': #386=IfcLocalPlacement(#178,#385),
 'Representation': #501=IfcProductDefinitionShape($,$,(#393,#498)),
 'Tag': '301422',
 'PredefinedType': 'NOTDEFINED'}

In [36]:
props = node_attributes(walls[0])
props

dict_keys(['Pset_WallCommon', 'Pset_EnvironmentalImpactIndicators', 'Pset_ReinforcementBarPitchOfWall', 'ADCM'])
has ADCM


{'global_id': '1syOIXGHX7Hg23ysDGAt9W',
 'name': 'Базовая стена:Внутренние - Кирпичная перегородка толщиной 250мм:301422',
 'is_a': 'IfcWall',
 'group_type': 'Базовая стена:Внутренние - Кирпичная перегородка толщиной 250мм',
 'ADCM_RD': 'AS',
 'ADCM_JobType': 'STP',
 'ADCM_DIN': '342',
 'ADCM_Level': 'L01',
 'ADCM_Title': '01p',
 'ADCM_Part': None,
 'coordinates': (1736.4932465136, 7367.87344148883, -100.0)}

In [25]:
from neo4j import GraphDatabase

group_driver = GraphDatabase.driver(
    "bolt://localhost:7688",
    auth=("neo4j", "23109900")
)

In [28]:
print(group_driver.verify_connectivity())

None


In [30]:
q = """
CREATE (n:Element)
SET n = $props
"""
group_driver.execute_query(q, props=props)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x0000029542B292D0>, keys=[])

In [ ]:
{'ADCM_RD': 'AS',
 'ADCM_JobType': 'STP', #wbs3
 'ADCM_DIN': '342', # wbs3_id
 'ADCM_Level': 'L01', # wbs2
 'ADCM_Title': '01p', # wbs1
 'id': 849}

In [12]:
ifcopenshell.util.element.get_psets(element, psets_only=True).keys()

dict_keys(['Pset_WallCommon', 'Pset_EnvironmentalImpactIndicators', 'Pset_ReinforcementBarPitchOfWall', 'ADCM'])

In [13]:
ifcopenshell.util.element.get_psets(element).keys()

dict_keys(['Pset_WallCommon', 'Qto_WallBaseQuantities', 'Pset_EnvironmentalImpactIndicators', 'Pset_ReinforcementBarPitchOfWall', 'ADCM'])

In [62]:
placement = door.ObjectPlacement
placement.get_info()
relative_placement = placement.RelativePlacement
relative_placement.get_info()
location = relative_placement.Location
location

{'id': 1199227,
 'type': 'IfcLocalPlacement',
 'PlacementRelTo': #1199222=IfcLocalPlacement(#334001,#1199221),
 'RelativePlacement': #1199226=IfcAxis2Placement3D(#3,$,$)}

In [12]:
coordinates = location.Coordinates
coordinates[0], coordinates[1], coordinates[2]
storeys = model.by_type("IfcBuildingStorey")
for storey in storeys:
    types = set(el.is_a() for el in storey.ContainsElements[0].RelatedElements)
    print(types)

(27819.99984492928, 19775.004971843486, 0.0)

In [9]:
# doors = list(filter(lambda el: el.is_a("IfcDoor"), storey.ContainsElements[0].RelatedElements))
elements = list(storey.ContainsElements[0].RelatedElements)
len(elements)

225

In [10]:
types = set(el.is_a() for el in elements)
types

{'IfcBuildingElementProxy', 'IfcWall'}

In [34]:
get_coordinates(doors[0])

(27819.99984492928, 19775.004971843486, 0.0)

In [40]:
p_doors = list(filter(lambda x: get_coordinates(x) != (0.0, 0.0, 0.0), doors))
len(p_doors)

44

In [41]:
get_coordinates(p_doors[1])

(-1.0950316209346056e-09, 0.0, 0.0)

In [43]:
get_coordinates(p_doors[2])

(-250.0, 200.0, 0.0)

In [44]:
get_coordinates(p_doors[3])

(250.0, -200.0, 0.0)

In [4]:
classes = ("IfcWall",
    "IfcDoor",
    "IfcSlab",
    "IfcCovering",
    "IfcBuildingElementProxy",
    "IfcStair",
    "IfcRailing",
    "IfcSpace",
    # "IfcBuildingStorey",
    "IfcWindow",
    # "IfcBuilding",
    "IfcCurtainWall")
classes

('IfcWall',
 'IfcDoor',
 'IfcSlab',
 'IfcCovering',
 'IfcBuildingElementProxy',
 'IfcStair',
 'IfcRailing',
 'IfcSpace',
 'IfcWindow',
 'IfcCurtainWall')

In [9]:
for i in classes:
    el = model.by_type(i)[0]
    print(i, hasattr(el, "ObjectPlacement"))
    print(get_coordinates(el))
    print()

IfcWall True
(14129.799844930327, 16000.004971844253, -100.0)

IfcDoor True
(27819.99984492928, 19775.004971843486, 0.0)

IfcSlab True
(0.0, 0.0, 0.0)

IfcCovering True
(0.0, 0.0, 0.0)

IfcBuildingElementProxy True
(66445.95098959892, 13990.0, 0.0)

IfcStair True
(0.0, 0.0, 0.0)

IfcRailing True
(15474.40472776368, 3663.0047277640956, -130.0)

IfcSpace True
(0.0, 0.0, 0.0)

IfcWindow True
(67297.49999999975, 147.49999999999963, 500.0)

IfcCurtainWall True
(0.0, 0.0, 0.0)



In [15]:
slab = model.by_type("IfcSlab")[0]
slab.ObjectPlacement.RelativePlacement.get_info()

{'id': 392059,
 'type': 'IfcAxis2Placement3D',
 'Location': #3=IfcCartesianPoint((0.,0.,0.)),
 'Axis': None,
 'RefDirection': None}

In [17]:
bool(slab.ObjectPlacement.RelativePlacement.Axis)

False

In [23]:
doors

NameError: name 'doors' is not defined